# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [20]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:
#import CSV file previously used
weather_data = "weather_data_df.csv"
weather_data_df = pd.read_csv(weather_data)
weather_data_df.head()

,Unnamed: 0,cities,lat,long,max_fahrenheit,humidity,cloudiness,wind_speed,country,date
0,0,Avarua,-21.21,-159.78,78.80,78,23,25.28,CK,1601425255
1,1,Noumea,-22.28,166.46,75.20,50,75,20.80,NC,1601425690
2,2,Chui,-33.70,-53.46,52.38,79,20,7.14,UY,1601425314
3,3,Vaini,-21.20,-175.20,80.60,83,90,14.99,TO,1601425692
4,4,Port Hardy,50.70,-127.42,66.20,77,20,1.86,CA,1601425692


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [22]:
city_coordinates = weather_data_df[["lat","long"]]
humidity = weather_data_df["humidity"]

In [10]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin':'0 auto 0 auto'
}
fig = gmaps.figure(layout = figure_layout)
markers = gmaps.marker_layer(city_coordinates)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [23]:
fig=gmaps.figure()
heat_layer= gmaps.heatmap_layer(city_coordinates, humidity, dissipating=False, max_intensity =10,point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
# Narrow down the DataFrame to find your ideal weather condition.
weather_data_df = weather_data_df.loc[weather_data_df['max_fahrenheit'] >70]
weather_data_df = weather_data_df.loc[weather_data_df['max_fahrenheit'] <80]

weather_data_df = weather_data_df.loc[weather_data_df['wind_speed'] <10]
weather_data_df = weather_data_df.loc[weather_data_df['cloudiness'] ==0]
weather_data_df

,Unnamed: 0,cities,lat,long,max_fahrenheit,humidity,cloudiness,wind_speed,country,date
43,43,Santiago del Estero,-27.80,-64.26,77.63,33,0,7.47,AR,1601425710
70,70,Caravelas,-17.71,-39.25,74.50,84,0,7.49,BR,1601425423
140,140,San Juan,-31.54,-68.54,70.14,16,0,7.63,AR,1601425401
151,151,Marsá Maţrūḩ,31.35,27.25,75.20,88,0,6.93,EG,1601425761
177,177,Sechelt,49.47,-123.77,72.00,69,0,2.80,CA,1601425775
233,233,Hīt,33.64,42.83,77.56,23,0,7.85,IQ,1601425811
309,309,Castaños,26.78,-101.42,78.01,20,0,3.00,MX,1601425850
313,313,Caculé,-14.50,-42.22,75.58,43,0,6.44,BR,1601425853
386,386,Viranşehir,37.24,39.76,75.25,23,0,7.54,TR,1601425887
397,397,Tezpur,26.63,92.80,78.53,86,0,2.93,IN,1601425893


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df = weather_data_df.dropna()

hotel_coord = zip(hotel_df["lat"],hotel_df["long"])
radius = 5000
target_type = 'lodging'

hotels = []

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


#for hotel_coord in hotel_coord:
for hotel_coord in hotel_coord:
    try:
        params = {
            'location': (f'{hotel_coord[0]},{hotel_coord[1]}'),
            'radius': radius,
            'type': target_type,
            'key':g_key
        }
        hotel_json = requests.get(url, params=params).json()
        hotels.append(hotel_json['results'][0]['name'])
    
    except IndexError:
        print(f"Error with hotel coordinates not found {hotel_coord[0]},{hotel_coord[1]}")
hotels    


['Hotel Urbano',
 'Pousada dos Navegantes',
 'Hotel Provincial',
 'Beau Site',
 'The Driftwood Inn',
 'Aboody Alhachem',
 'Bomberos voluntarios castaños coahuila',
 'Radar Palace Hotel',
 'Viranşehir Öğretmenevi',
 'Hotel Kanyapur',
 'فروشگاه موبایل ترکمانی',
 'Ambey mobail care',
 'رابطة الفنانين الفلسطينيين',
 'محمد هادي ال فطيح',
 'CABAÑAS DORALIA']

In [17]:
#drop_missing_hotel = hotel_df.drop([31])
# drop_missing_hotel["Hotel Name"] = hotels
# drop_missing_hotel
hotel_df["Hotel Name"] = hotels
hotel_df

,Unnamed: 0,cities,lat,long,max_fahrenheit,humidity,cloudiness,wind_speed,country,date,Hotel Name
43,43,Santiago del Estero,-27.80,-64.26,77.63,33,0,7.47,AR,1601425710,Hotel Urbano
70,70,Caravelas,-17.71,-39.25,74.50,84,0,7.49,BR,1601425423,Pousada dos Navegantes
140,140,San Juan,-31.54,-68.54,70.14,16,0,7.63,AR,1601425401,Hotel Provincial
151,151,Marsá Maţrūḩ,31.35,27.25,75.20,88,0,6.93,EG,1601425761,Beau Site
177,177,Sechelt,49.47,-123.77,72.00,69,0,2.80,CA,1601425775,The Driftwood Inn
233,233,Hīt,33.64,42.83,77.56,23,0,7.85,IQ,1601425811,Aboody Alhachem
309,309,Castaños,26.78,-101.42,78.01,20,0,3.00,MX,1601425850,Bomberos voluntarios castaños coahuila
313,313,Caculé,-14.50,-42.22,75.58,43,0,6.44,BR,1601425853,Radar Palace Hotel
386,386,Viranşehir,37.24,39.76,75.25,23,0,7.54,TR,1601425887,Viranşehir Öğretmenevi
397,397,Tezpur,26.63,92.80,78.53,86,0,2.93,IN,1601425893,Hotel Kanyapur


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
